# Classification task

This notebook covers the stages of building the classifier model. These will include:
- Original vs enhanced in a general comparison
- Selection comparsion
- Hyperparameter tuning
- Final training

The following libraries are used.

In [51]:
# Data manipulation
import pandas as pd

# Scikit-learn
from sklearn.metrics import recall_score, precision_score

# Keras
import keras
import keras.backend as K
import keras_metrics
from keras import Model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.models import load_model

# Pretrained architectures
from keras.applications import *

In [ ]:
keras.__version__

## Global functions and definitions

### Model modifiers

#### Layer freezing without batch normalisation

In [ ]:
def batch_norm_unfrozen(model):
    for layer in model.layers[:-4]:    
        if str(layer.__class__)[-20:-2] != 'BatchNormalization':  # Keep Batch normalization layers trainable
            layer.trainable = False

#### Define top layer architecture

In [ ]:
def add_top(base_model, dense=256, active='relu', dropout=0.5):
    base_out = base_model.output
    base_out = GlobalAveragePooling2D()(base_out)
    base_out = Dense(dense, activation=active)(base_out)
    base_out = Dropout(dropout)(base_out)
    base_out = Dense(1, activation='sigmoid')(base_out)

    return Model(base_model.input, base_out)

### Callbacks

#### Tensorboard

In [ ]:
def set_tensorboard(name, directory='logs/cls/'):
    return TensorBoard(log_dir=directory + str(name))

#### Early stop

In [ ]:
def set_early_stop(patience=50, verbose=0):
    return EarlyStopping(monitor='val_loss', 
                         patience=patience,
                         mode='min',
                         verbose=verbose)

#### Learning rate plateau

In [ ]:
def set_lr_plateau(factor=0.5, patience=25, verbose=0):
    return ReduceLROnPlateau(monitor='val_loss', 
                             factor=factor, 
                             patience=patience, 
                             mode='min',
                             verbose=verbose)

### Custom metric

In [ ]:
def value_retained(y_true, y_pred):
    true_pos = np.sum([pred and true for true, pred in zip(y_true, y_pred)])
    true_neg = np.sum([not pred and not true for true, pred in zip(y_true, y_pred)])
    false_pos = np.sum([pred > true for true, pred in zip(y_true, y_pred)])
    
    return (true_pos + true_neg + false_pos*0.64)/len(y_true)

### Data generators

Define batch and image size.

In [52]:
BATCH_SIZE = 10
IMAGE_SHAPE = (128, 128, 3)

#### Image data generator for full training set

In [53]:
full_train_datagen = ImageDataGenerator(rotation_range=90, 
                                   shear_range=0.3, 
                                   zoom_range=0.3, 
                                   horizontal_flip=True, 
                                   vertical_flip=True, 
                                   rescale=1./255)

#### Image data generator for validation sets

Define augmentations and validation set of image data generator.

In [54]:
train_datagen = ImageDataGenerator(rotation_range=90, 
                                   shear_range=0.3, 
                                   zoom_range=0.3, 
                                   horizontal_flip=True, 
                                   vertical_flip=True, 
                                   rescale=1./255,
                                   validation_split=0.197)

#### Image data generator for test set

In [55]:
test_datagen = ImageDataGenerator(rescale=1./255)

#### Import dataframes

Drop percentage for the classification task.

In [56]:
# Validation sets for model comparisons
df_train = []
for idx in range(10):
    df_train.append(pd.read_csv('dataframes/train_'+ str(idx) + '.csv').drop(['percentage'], axis=1))

# Full training set for final model training
full_df_train = pd.read_csv('dataframes/train.csv').drop(['percentage'], axis=1)
   
# Test set for final evaluation
df_test = pd.read_csv('dataframes/test.csv').drop(['percentage'], axis=1)

#### Create data generators for the original images

In [57]:
# Validation set generators
train_gen = []
valid_gen = []

for idx in range(10):
    train_gen.append(train_datagen.flow_from_dataframe(
                        dataframe=df_train[idx],
                        directory='data/crop', 
                        x_col='filename', 
                        y_col='rbr', 
                        has_ext=True, 
                        target_size=IMAGE_SHAPE[:2], 
                        color_mode='rgb', 
                        classes=None, 
                        class_mode='binary', 
                        batch_size=BATCH_SIZE, 
                        shuffle=True, 
                        seed=42, 
                        subset='training')
                    )

    valid_gen.append(train_datagen.flow_from_dataframe(
                        dataframe=df_train[idx], 
                        directory='data/crop', 
                        x_col='filename', 
                        y_col='rbr', 
                        has_ext=True, 
                        target_size=IMAGE_SHAPE[:2], 
                        color_mode='rgb', 
                        classes=None, 
                        class_mode='binary', 
                        batch_size=BATCH_SIZE, 
                        shuffle=True, 
                        seed=42, 
                        subset='validation')
                    )

# Full training set generator
full_train_gen = full_train_datagen.flow_from_dataframe(
                        dataframe=full_df_train,
                        directory='data/crop', 
                        x_col='filename', 
                        y_col='rbr', 
                        has_ext=True, 
                        target_size=IMAGE_SHAPE[:2], 
                        color_mode='rgb', 
                        classes=None, 
                        class_mode='binary', 
                        batch_size=BATCH_SIZE, 
                        shuffle=True, 
                        seed=42)

# Test set generator
test_gen = test_datagen.flow_from_dataframe(
                        dataframe=df_test,
                        directory='data/crop', 
                        x_col='filename', 
                        y_col='rbr', 
                        has_ext=True, 
                        target_size=IMAGE_SHAPE[:2], 
                        color_mode='rgb', 
                        classes=None, 
                        class_mode=None, 
                        batch_size=BATCH_SIZE, 
                        shuffle=False, 
                        seed=42)

Found 650 images belonging to 2 classes.
Found 160 images belonging to 2 classes.
Found 650 images belonging to 2 classes.
Found 160 images belonging to 2 classes.
Found 650 images belonging to 2 classes.
Found 160 images belonging to 2 classes.
Found 650 images belonging to 2 classes.
Found 160 images belonging to 2 classes.
Found 650 images belonging to 2 classes.
Found 160 images belonging to 2 classes.
Found 650 images belonging to 2 classes.
Found 160 images belonging to 2 classes.
Found 650 images belonging to 2 classes.
Found 160 images belonging to 2 classes.
Found 650 images belonging to 2 classes.
Found 160 images belonging to 2 classes.
Found 650 images belonging to 2 classes.
Found 160 images belonging to 2 classes.
Found 650 images belonging to 2 classes.
Found 160 images belonging to 2 classes.
Found 810 images belonging to 2 classes.
Found 200 images.


Create data generators for the equalised images.

In [ ]:
equ_train_gen = []
equ_valid_gen = []

for idx in range(10):
    equ_train_gen.append(train_datagen.flow_from_dataframe(
                            dataframe=df_train[idx],
                            directory='data/equ_crop', 
                            x_col='filename', 
                            y_col='rbr', 
                            has_ext=True, 
                            target_size=IMAGE_SHAPE[:2], 
                            color_mode='rgb', 
                            classes=None, 
                            class_mode='binary', 
                            batch_size=BATCH_SIZE, 
                            shuffle=True, 
                            seed=42, 
                            subset='training')
                        )

    equ_valid_gen.append(train_datagen.flow_from_dataframe(dataframe=df_train[idx], 
                            directory='data/equ_crop', 
                            x_col='filename', 
                            y_col='rbr', 
                            has_ext=True, 
                            target_size=IMAGE_SHAPE[:2], 
                            color_mode='rgb', 
                            classes=None, 
                            class_mode='binary', 
                            batch_size=BATCH_SIZE, 
                            shuffle=True, 
                            seed=42, 
                            subset='validation')
                        )

## Original vs enhanced images

In [ ]:
comp_pretrained = [DenseNet121,
                   DenseNet169,
                   DenseNet201,
                   InceptionResNetV2,
                   InceptionV3,
                   MobileNet,
                   MobileNetV2,
                   NASNetMobile,
                   NASNetLarge,
                   ResNet50,
                   VGG16,
                   VGG19,
                   Xception]

models = ['DenseNet121',
          'DenseNet169',
          'DenseNet201',
          'InceptionResNetV2',
          'InceptionV3',
          'MobileNet',
          'MobileNetV2',
          'NASNetMobile',
          'NASNetLarge',
          'ResNet50',
          'VGG16',
          'VGG19',
          'Xception']

In [ ]:
def compare_original(architectures, start_model, start_fold, n_folds):

    count = 0
    
    for idx, architecture in enumerate(architectures):
        if idx >= start_model:
            for fold in range(start_fold, 10):
                base_model = architecture(weights='imagenet', 
                                          include_top=False, 
                                          input_shape=IMAGE_SHAPE)

                model_name = models[idx]

                batch_norm_unfrozen(base_model)
                model = add_top(base_model)

                model.compile( 
                    loss='binary_crossentropy', 
                    optimizer=optimizers.nadam(), 
                    metrics=[keras_metrics.precision(), keras_metrics.recall()]
                )

                train_gen[fold].reset()
                valid_gen[fold].reset()

                STEP_SIZE_TRAIN=train_gen[fold].n//train_gen[fold].batch_size
                STEP_SIZE_VALID=valid_gen[fold].n//valid_gen[fold].batch_size

                history = (model.fit_generator(generator=train_gen[fold],
                                                steps_per_epoch=STEP_SIZE_TRAIN,
                                                validation_data=valid_gen[fold],
                                                validation_steps=STEP_SIZE_VALID,
                                                epochs=100,
                                                verbose=0,
                                                callbacks=[set_tensorboard('comp/' + model_name + '_' + str(fold))]))
                del model
                del base_model

                if n_folds <= count:
                    return
                else:
                    count = count + 1

In [ ]:
def compare_equalised(architectures, start_model, start_fold, n_folds):

    count = 0
    
    for idx, architecture in enumerate(architectures):
        if idx >= start_model:
            for fold in range(start_fold, 10):
                base_model = architecture(weights='imagenet', 
                                          include_top=False, 
                                          input_shape=IMAGE_SHAPE)

                model_name = models[idx]

                batch_norm_unfrozen(base_model)
                model = add_top(base_model)

                model.compile( 
                    loss='binary_crossentropy', 
                    optimizer=optimizers.nadam(), 
                    metrics=[keras_metrics.precision(), keras_metrics.recall()]
                )

                equ_train_gen[fold].reset()
                equ_valid_gen[fold].reset()

                STEP_SIZE_TRAIN=equ_train_gen[fold].n//equ_train_gen[fold].batch_size
                STEP_SIZE_VALID=equ_valid_gen[fold].n//equ_valid_gen[fold].batch_size

                history = (model.fit_generator(generator=equ_train_gen[fold],
                                                steps_per_epoch=STEP_SIZE_TRAIN,
                                                validation_data=equ_valid_gen[fold],
                                                validation_steps=STEP_SIZE_VALID,
                                                epochs=100,
                                                verbose=0,
                                                callbacks=[set_tensorboard('enchanced/' + model_name + '_' + str(fold))]))
                del model
                del base_model

                if n_folds <= count:
                    return
                else:
                    count = count + 1

The functions above iterate through the list of architecture passed. It then:
- downloads the architecture
- freezes the layers other than batch normalisation 
- adds the top layers
- compiles the model
- trains the model

The function layer is very similar for all comparisons run, but not identical. The function can be passed a starting point in case of interuption. A limit to the number of iterations can also be set because there is a memory leak in the process that slows down, and eventually stops, the training process. It was found that after 6 iterations, the process slowed down exponentially. Running one of the full processes takes approximately 65 hours, depending on the available computational power (GPU enabled).

In [ ]:
compare_original(comp_pretrained, 0, 0, 6)

In [ ]:
compare_equalised(comp_pretrained, 0, 0, 6)

The results can then be analysed using Tensorboard or downloaded to csv and imported. The results of the original and enhanced images can be compared and a subset of model can be selected.

## Selection comparison

In [ ]:
select_pretrained = [DenseNet121,
                    DenseNet169,
                    DenseNet201,
                    MobileNet,
                    VGG16]

models = ['DenseNet121',
        'DenseNet169',
        'DenseNet201',
        'MobileNet',
        'VGG16']

In [ ]:
def compare_selected(architectures, start_model, start_fold, n_folds):
    
    count = 0
    
    for idx, architecture in enumerate(architectures):
        if idx >= start_model:
            for fold in range(start_fold, 10):
                base_model = architecture(weights='imagenet', 
                                          include_top=False, 
                                          input_shape=IMAGE_SHAPE)

                model_name = models[idx]

                batch_norm_unfrozen(base_model)
                model = add_top(base_model)

                model.compile( 
                    loss='binary_crossentropy', 
                    optimizer=optimizers.nadam(), 
                    metrics=[keras_metrics.precision(), keras_metrics.recall()]
                )

                train_gen[fold].reset()
                valid_gen[fold].reset()

                STEP_SIZE_TRAIN=train_gen[fold].n//train_gen[fold].batch_size
                STEP_SIZE_VALID=valid_gen[fold].n//valid_gen[fold].batch_size

                history = (model.fit_generator(generator=train_gen[fold],
                                                steps_per_epoch=STEP_SIZE_TRAIN,
                                                validation_data=valid_gen[fold],
                                                validation_steps=STEP_SIZE_VALID,
                                                epochs=300,
                                                verbose=0,
                                                callbacks=[set_tensorboard('select/' + model_name + '_' + str(fold)),
                                                           set_lr_plateau(),
                                                           set_early_stop()]))

                del model
                del base_model
                
                if n_folds <= count:
                    return
                else:
                    count = count + 1
    return

In [ ]:
compare_selected(select_pretrained, 0, 0, 6)

## Hyperparameter tuning

In [ ]:
def grid_search(dense, active, start_dense=0, start_active=0, start_fold=0, n_folds=6):
    
    count = 0
    
    for d, units in enumerate(dense):
        if d >= start_dense:
            for a, func in enumerate(active):
                if a >= start_active:
                        for fold in range(start_fold, 10):
                            base_model = DenseNet201(weights='imagenet',
                                                     include_top=False,
                                                     input_shape=IMAGE_SHAPE)

                            batch_norm_unfrozen(base_model)
                            model = add_top(base_model, dense=units, active=func)

                            model.compile( 
                                loss='binary_crossentropy', 
                                optimizer=optimizers.nadam(), 
                                metrics=[keras_metrics.precision(), keras_metrics.recall()]
                            )

                            # TRAIN
                            train_gen[fold].reset()
                            valid_gen[fold].reset()

                            STEP_SIZE_TRAIN=train_gen[fold].n//train_gen[fold].batch_size
                            STEP_SIZE_VALID=valid_gen[fold].n//valid_gen[fold].batch_size
                            history_pretrained = model.fit_generator(generator=train_gen[fold],
                                                                     steps_per_epoch=STEP_SIZE_TRAIN,
                                                                     validation_data=valid_gen[fold],
                                                                     validation_steps=STEP_SIZE_VALID,
                                                                     epochs=120,
                                                                     verbose=0,
                                                                     callbacks=[set_tensorboard('optim/DenseNet201_' + str(units) + '_' + func + '_' + str(fold))])

                            del model
                            del base_model
                            
                            if n_folds <= count:
                                return
                            else:
                                count = count + 1


In [ ]:
# Range of dense units
dense = [16, 64, 256, 1024]

# Activation functions
active = ['relu', 'elu']

grid_search(dense, active)

## Final training

The final training uses the validation curves to establish a optimal number of epochs to train for.

In [ ]:
def training_time(start_fold):
    
    count = 0
    
    for fold in range(start_fold, 10):
        base_model = DenseNet201(weights='imagenet',
                                 include_top=False,
                                 input_shape=IMAGE_SHAPE)

        batch_norm_unfrozen(base_model)
        model = add_top(base_model, dense=32, active='elu')

        model.compile( 
            loss='binary_crossentropy', 
            optimizer=optimizers.nadam(), 
            metrics=[keras_metrics.precision(), keras_metrics.recall()]
        )

        # TRAIN
        train_gen[fold].reset()
        valid_gen[fold].reset()

        STEP_SIZE_TRAIN=train_gen[fold].n//train_gen[fold].batch_size
        STEP_SIZE_VALID=valid_gen[fold].n//valid_gen[fold].batch_size
        history_pretrained = model.fit_generator(generator=train_gen[fold],
                                                 steps_per_epoch=STEP_SIZE_TRAIN,
                                                 validation_data=valid_gen[fold],
                                                 validation_steps=STEP_SIZE_VALID,
                                                 epochs=200,
                                                 verbose=0,
                                                 callbacks=[set_tensorboard('final/DenseNet201_' + str(fold))])

        del model
        del base_model
        
        if n_folds <= count:
            return
        else:
            count = count + 1

In [ ]:
training_time(0)

This function includes a learning rate plateau callback which decreases the learning rate if the validation loss plateaus. This is a mistake. It's not possible to use when training the final model since a validation set is not used. All learning rate decreased for all 10 folds on the same epoch, so the final model was trained in stages, decreasing the learning rate between each stage.

In [ ]:
def final_model(model, epochs, i_epoch, learning_rate):
    model.compile( 
        loss='binary_crossentropy', 
        optimizer=optimizers.nadam(lr=learning_rate), 
        metrics=[keras_metrics.precision(), keras_metrics.recall()]
    )

    full_train_gen.reset()

    STEP_SIZE_TRAIN=full_train_gen.n//full_train_gen.batch_size
    history_pretrained = model.fit_generator(generator=full_train_gen,
                                             steps_per_epoch=STEP_SIZE_TRAIN,
                                             epochs=epochs,
                                             initial_epoch=i_epoch,
                                             verbose=0,
                                             callbacks=[set_tensorboard('final/DenseNet201_final')])

    return model

Train the model and save it.

In [ ]:
base_model = DenseNet201(weights='imagenet',
                         include_top=False,
                         input_shape=IMAGE_SHAPE)

batch_norm_unfrozen(base_model)
model = add_top(base_model, dense=32, active='elu', dropout=0.4)

model = final_model(model, 100, 0, 0.002)

model.save('models/classification_final_test')

## Evaluate the model

In [58]:
model = load_model('models/classification_final')

Evaluate the model using the test set image data generator.

In [63]:
test_gen.reset()

predictions = model.predict_generator(generator=test_gen, steps=test_gen.n//BATCH_SIZE, verbose=1)

y_pred = predictions > 0.5
y_true = list(df_test['rbr'].astype(int))

recall = recall_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
f1 = 2*(precision*recall)/(precision+recall)
val = value_retained(y_true, y_pred)

print('Pretrained model recall: {}'.format(recall))
print('Pretrained model precision: {}'.format(precision))
print('Pretrained model F1 score: {}'.format(f1))
print('Pretrained model value score: {}'.format(val))

20/20 [==============================] - 3s 150ms/step
Pretrained model recall: 1.0
Pretrained model precision: 0.9523809523809523
Pretrained model F1 score: 0.975609756097561
Pretrained model value score: 0.991
